In [1]:

!pip uninstall opencv-python -y
!pip uninstall opencv-contrib-python -y
!pip uninstall opencv-python-headless -y
!pip install opencv-python==4.5.4.60
!pip install opencv-python-headless==4.5.4.60
!pip install opencv-contrib-python==4.5.4.60
!pip install imutils
!pip install dlib
!pip install tensorflow_addons
!pip install autocrop 

import numpy as np
import pandas as pd
import random as rd
import cv2
import os
import math

import matplotlib.pyplot as plt

from enum import Enum
from glob import glob
from functools import partial

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow_addons.layers import InstanceNormalization

import sys
from pathlib import Path
import PIL
from autocrop import Cropper

import imutils
import dlib
#from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
from imutils.face_utils import FACIAL_LANDMARKS_IDXS

Found existing installation: opencv-python 4.5.4.60
Uninstalling opencv-python-4.5.4.60:
  Successfully uninstalled opencv-python-4.5.4.60
Found existing installation: opencv-contrib-python 4.5.4.60
Uninstalling opencv-contrib-python-4.5.4.60:
  Successfully uninstalled opencv-contrib-python-4.5.4.60
Found existing installation: opencv-python-headless 4.5.4.60
Uninstalling opencv-python-headless-4.5.4.60:
  Successfully uninstalled opencv-python-headless-4.5.4.60
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25858 sha256=7f047c9bb99ddae22e0f20fd2103e46b0dba60be77977b4b73dc597feb2e9d1b
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0

In [2]:
from imutils.face_utils.helpers import FACIAL_LANDMARKS_68_IDXS
from imutils.face_utils.helpers import FACIAL_LANDMARKS_5_IDXS
from imutils.face_utils.helpers import shape_to_np
import numpy as np
import cv2

class FaceAligner:
	def __init__(self, predictor, desiredLeftEye=(0.35, 0.35),
		desiredFaceWidth=512, desiredFaceHeight=None):
		# store the facial landmark predictor, desired output left
		# eye position, and desired output face width + height
		self.predictor = predictor
		self.desiredLeftEye = desiredLeftEye
		self.desiredFaceWidth = desiredFaceWidth
		self.desiredFaceHeight = desiredFaceHeight

		# if the desired face height is None, set it to be the
		# desired face width (normal behavior)
		if self.desiredFaceHeight is None:
			self.desiredFaceHeight = self.desiredFaceWidth

	def align(self, image, gray, rect):
		# convert the landmark (x, y)-coordinates to a NumPy array
		shape = self.predictor(gray, rect)
		shape = shape_to_np(shape)
		
		#simple hack ;)
		if (len(shape)==68):
			# extract the left and right eye (x, y)-coordinates
			(lStart, lEnd) = FACIAL_LANDMARKS_68_IDXS["left_eye"]
			(rStart, rEnd) = FACIAL_LANDMARKS_68_IDXS["right_eye"]
		else:
			(lStart, lEnd) = FACIAL_LANDMARKS_5_IDXS["left_eye"]
			(rStart, rEnd) = FACIAL_LANDMARKS_5_IDXS["right_eye"]
			
		leftEyePts = shape[lStart:lEnd]
		rightEyePts = shape[rStart:rEnd]

		# compute the center of mass for each eye
		leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
		rightEyeCenter = rightEyePts.mean(axis=0).astype("int")

		# compute the angle between the eye centroids
		dY = rightEyeCenter[1] - leftEyeCenter[1]
		dX = rightEyeCenter[0] - leftEyeCenter[0]
		angle = np.degrees(np.arctan2(dY, dX)) - 180

		# compute the desired right eye x-coordinate based on the
		# desired x-coordinate of the left eye
		desiredRightEyeX = 1.0 - self.desiredLeftEye[0]

		# determine the scale of the new resulting image by taking
		# the ratio of the distance between eyes in the *current*
		# image to the ratio of distance between eyes in the
		# *desired* image
		dist = np.sqrt((dX ** 2) + (dY ** 2))
		desiredDist = (desiredRightEyeX - self.desiredLeftEye[0])
		desiredDist *= self.desiredFaceWidth
		scale = desiredDist / dist

		# compute center (x, y)-coordinates (i.e., the median point)
		# between the two eyes in the input image
		eyesCenter = (int((leftEyeCenter[0] + rightEyeCenter[0]) // 2),
			(int(leftEyeCenter[1] + rightEyeCenter[1]) // 2))

		# grab the rotation matrix for rotating and scaling the face
		M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

		# update the translation component of the matrix
		tX = self.desiredFaceWidth * 0.5
		tY = self.desiredFaceHeight * self.desiredLeftEye[1]
		M[0, 2] += (tX - eyesCenter[0])
		M[1, 2] += (tY - eyesCenter[1])

		# apply the affine transformation
		(w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
		output = cv2.warpAffine(image, M, (w, h),
			flags=cv2.INTER_CUBIC)

		# return the aligned face
		return output

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../input/models/landmarks.dat')
fa = FaceAligner(predictor, desiredFaceWidth=512)

def align_images(path):
    DIR = os.listdir(path)
    for img in DIR :
        print(img)
        image = cv2.imread(path+img)
        image = imutils.resize(image, width=800)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 2)

# loop over the face detections
        for rect in rects:
	# extract the ROI of the *original* face, then align the face
	# using facial landmarks
            (x, y, w, h) = rect_to_bb(rect)
            faceOrig = imutils.resize(image[y:y + h, x:x + w], width=512)
            faceAligned = fa.align(image, gray, rect)
            cv2.imwrite("./aligned-"+img,faceAligned)
            print(img)

In [4]:
PATH = "../input/kpop-idols/"
align_images(PATH)

wonyoung-izone-bloomiz-uhdp.jpg
wonyoung-izone-bloomiz-uhdp.jpg
wallpaperflare.com_wallpaper.jpg
wallpaperflare.com_wallpaper.jpg
346250-BLACKPINK-Kpop-K-pop-Girls-Jisoo-K.jpg
346250-BLACKPINK-Kpop-K-pop-Girls-Jisoo-K.jpg
DzP3Hs6X4AAflHS.jpg
DzP3Hs6X4AAflHS.jpg
DzP3Hs6X4AAflHS.jpg
DzP3Hs6X4AAflHS.jpg
DzP3Hs6X4AAflHS.jpg
DzP3Hs6X4AAflHS.jpg
isa-stayc-we-need-love-4k-wallpaper-uhdpape.jpg
isa-stayc-we-need-love-4k-wallpaper-uhdpape.jpg
hong-eunchae-le-sserafim-4k-wallpaper-uhdpaper.jpg
hong-eunchae-le-sserafim-4k-wallpaper-uhdpaper.jpg
wallpaperflare.com_wallpaper (12).jpg
wallpaperflare.com_wallpaper (12).jpg
wallpaperflare.com_wallpaper (11).jpg
wallpaperflare.com_wallpaper (11).jpg
wallpaperflare.com_wallpaper (9).jpg
wallpaperflare.com_wallpaper (9).jpg
wallpaperflare.com_wallpaper (3).jpg
wallpaperflare.com_wallpaper (3).jpg
hyunjin-loona-flip-that-4k-wallpaper-uhd.jpg
hyunjin-loona-flip-that-4k-wallpaper-uhd.jpg
viviz-members-profile-4k-wallpaper-uhdpap.jpg
viviz-members-profile-4k